In [1]:
import pandas as pd
from datetime import datetime, timedelta
from os import path
import glob
import matplotlib.pyplot as plt
import numpy as np
import re

In [2]:
date = datetime.today() - timedelta(days = 1)
min_date = datetime(year = 2020, month = 1, day = 22)

data_dir = "data"

while date >= min_date:
    date_formatted = date.strftime("%m-%d-%Y")
    filepath = f"{data_dir}/{date_formatted}.csv"
    if not path.isfile(filepath):    
        url = f"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_formatted}.csv"
        dataframe = pd.read_csv(url)
        dataframe.to_csv(filepath, index = False)
        print(f"Saved {filepath}")
    else:
        print(f"Skipped {filepath}")
    date = date - timedelta(days = 1)

Saved data/04-20-2020.csv
Saved data/04-19-2020.csv
Skipped data/04-18-2020.csv
Skipped data/04-17-2020.csv
Skipped data/04-16-2020.csv
Skipped data/04-15-2020.csv
Skipped data/04-14-2020.csv
Skipped data/04-13-2020.csv
Skipped data/04-12-2020.csv
Skipped data/04-11-2020.csv
Skipped data/04-10-2020.csv
Skipped data/04-09-2020.csv
Skipped data/04-08-2020.csv
Skipped data/04-07-2020.csv
Skipped data/04-06-2020.csv
Skipped data/04-05-2020.csv
Skipped data/04-04-2020.csv
Skipped data/04-03-2020.csv
Skipped data/04-02-2020.csv
Skipped data/04-01-2020.csv
Skipped data/03-31-2020.csv
Skipped data/03-30-2020.csv
Skipped data/03-29-2020.csv
Skipped data/03-28-2020.csv
Skipped data/03-27-2020.csv
Skipped data/03-26-2020.csv
Skipped data/03-25-2020.csv
Skipped data/03-24-2020.csv
Skipped data/03-23-2020.csv
Skipped data/03-22-2020.csv
Skipped data/03-21-2020.csv
Skipped data/03-20-2020.csv
Skipped data/03-19-2020.csv
Skipped data/03-18-2020.csv
Skipped data/03-17-2020.csv
Skipped data/03-16-2020.

In [3]:
data_files = glob.glob(f"{data_dir}/*.csv")

daily_dataframes = []
p = re.compile(r'(\d{2}-\d{2}-\d{2})')

for file in data_files:
    daily_dataframe = pd.read_csv(file)
    daily_dataframe = daily_dataframe.rename({"Last_Update": "Last Update", "Country_Region": "Country/Region"}, axis=1)
    daily_dataframe["Last Update"] = p.search(file).group()
    daily_dataframes.append(daily_dataframe)

combined_data = pd.concat(daily_dataframes, axis=0, ignore_index=True, sort=False)
combined_data = combined_data.sort_values(by="Last Update", ascending=False)
combined_data = combined_data.filter(["Country/Region", "Last Update", "Confirmed", "Deaths", "Recovered"], axis=1)
combined_data["Last Update"] = combined_data["Last Update"].apply(lambda x: datetime.strptime(x, "%m-%d-%y"))
combined_data["Country/Region"] = combined_data["Country/Region"].replace("United Kingdom", "UK")
combined_data["Country/Region"] = combined_data["Country/Region"].replace("Mainland China", "China")

In [4]:
countries_selected = ["UK", "Italy"]
filtered_data = combined_data[combined_data["Country/Region"].isin(countries_selected)]
grouped_data = filtered_data.groupby(by=["Last Update", "Country/Region"]).sum()
print(grouped_data.tail(10))

                            Confirmed   Deaths  Recovered
Last Update Country/Region                               
2020-04-16  Italy            168941.0  22170.0    40164.0
            UK               104145.0  13759.0      375.0
2020-04-17  Italy            172434.0  22745.0    42727.0
            UK               109769.0  14607.0      394.0
2020-04-18  Italy            175925.0  23227.0    44927.0
            UK               115314.0  15498.0      414.0
2020-04-19  Italy            178972.0  23660.0    47055.0
            UK               121172.0  16095.0      436.0
2020-04-20  Italy            181228.0  24114.0    48877.0
            UK               125856.0  16550.0      446.0


In [7]:
fig, ax = plt.subplots(2, 1, figsize=(10,15))
clean_data = grouped_data.unstack()
clean_data.plot(kind="line", ax=ax[0], lw=2)
clean_data.plot(kind="line", ax=ax[1], lw=2)
ax[1].set_yscale("log")
plt.savefig("plots/line.png")
plt.close(fig)